google sheet template properties?


In [ ]:
var importer = require('../Core');
var getDataSheet = importer.import('google sheet array objects');
var renderRows = importer.import('google sheet layout template');
var getRows = importer.import('get worksheet rows');

function getTemplateProperties(value, properties, templates) {
    var returnObj = {};
    var promise;
    if(typeof properties[value + '-data'] != 'undefined') {
        promise = Promise.resolve(properties[value + '-data']);
    } else if(templates[value].data) {
        promise = getDataSheet(templates[value].data);
    } else {
        promise = Promise.resolve(false);
    }
    
    return promise
        .then(data => {
            properties[value + '-data'] = data;
            returnObj[value + '-data'] = data;
            var templatePromise;
            if(typeof properties[value] != 'undefined') {
                templatePromise = Promise.resolve(properties[value]);
            } else {
                templatePromise = getSections(value, properties, templates)
                    .then(rows => renderRows(value, rows, properties));
            }
            return templatePromise;
        })
        .then(template => {
            properties[value] = template;
            returnObj[value] = template;
            return returnObj;
        });
}

// must do this up front so we can process all data
function getSections(layoutTitle, properties, templates) {
    var worksheet = templates[layoutTitle].template;
    if(!worksheet) {
        return Promise.resolve(false);
    }
    console.log('Reading sheet: ' + worksheet.properties.title + ' - '
                + worksheet.properties.gridProperties.rowCount);
    return getRows(worksheet)
        .then(rows => {
            var sections = rows.reduce((obj, row, i) => {
                var subsections = row.reduce((obj, c, j) => {
                    var value = row[j + 1];
                    if(c === ':render') {
                        if(!properties[':render']) {
                            properties[':render'] = [];
                        }
                        properties[':render'][properties[':render'].length] = value;
                    }
                    if(c === ':section') {
                        if(typeof templates[value] != 'undefined') {
                            obj[value] = getTemplateProperties(value, properties, templates);
                        } else {
                            console.log(`Section ${value} not found!`)
                        }
                    }
                    return obj;
                }, obj);
                return obj;
            }, {});
            return Promise.all(Object.values(sections)).then(() => rows);
        })
}

module.exports = getTemplateProperties;

google sheet layout template?

Convert a google sheet page to an HTML template



In [ ]:
var importer = require('../Core');
var Remarkable = require('remarkable');
var md = new Remarkable({html: true, xhtmlOut: true, breaks: true});

function getDataClasses(c, data) {
    // get classes from mustache vars used with supplied data
    var dataKeys = (data || []).reduce((keys, cur) => {
        // get all keys in data
        return keys.concat(Object.keys(cur));
    }, []).filter((k, h, a) => a.indexOf(k) == h);
    var dataClasses = dataKeys
        .filter(k => ('\\{\\{' + k + '\\}\\}', 'ig').match(c))
        .map(k => 'val-' + k)
        .join(' ')
    return dataClasses;
}

function renderRows(layoutTitle, rows, properties) {
    // set object properties for mustache template
    var html = rows.reduce((arr, row, i) => {
        var rowsHtml = row.reduce((arr, c, j) => {
            var dataClasses = getDataClasses(c, properties[layoutTitle + '-data']);
            if(c.substr(0, 1) === ':') {
                // use subsequent column for property values
                var value = row[j + 1];
                if(c.substr(1).toLowerCase() === 'section') {
                    arr[arr.length] = `
<div class="${dataClasses} cell-${j} section-${value}">
${properties[value]}
</div>
`;
                } else {
                    properties[c.substr(1)] = value;
                }
            // render markdown content if it is not the value for the previous property
            } else if(j == 0 || j >= 1 && row[j - 1]
                      && row[j - 1].substr(0, 1) !== ':') {
                var mdHtml = md.render(c);
                // if all markdown did was insert a paragraph and line break, use value instead
                if(mdHtml.replace(/<\/?p>|\s+/ig, '') == c.replace(/<\/?p>|\s+/ig, '')) {
                    mdHtml = c;
                }
                arr[arr.length] = `
<div class="${dataClasses} cell-${j}">
${mdHtml}
</div>
`;
            }
            return arr;
        }, []);

        // render mustache templates
        if(rowsHtml.length > 0) {
            var rowTemplate = `
<div class="${properties['class'] || ''} col-${rowsHtml.length}">
${rowsHtml.join('')}
</div>
`;
            arr[arr.length] = rowTemplate;
        }

        return arr;
    }, []);

    // render mustache page template
    return html.join('');
}

module.exports = renderRows;

output google sheet template?

Save the generated template to an HTML file, wrapping it in a base template


In [ ]:
var fs = require('fs');
var path = require('path');
var Mustache = require('mustache');

function safeName(val, render) {
    return render(val).replace(/[^a-z\-]/ig, '-').substr(0, 40);
}

function wrapTemplate(layoutTitle, html, properties) {
    properties['safeName'] = () => safeName;
    // TODO: turn this in to an array/section?
    var script = '', banner = '', domain = '';
    if(typeof properties['stylesheet'] == 'undefined') {
        properties['stylesheet'] = '/illuminati-style.css';
    }
    if(typeof properties['script'] != 'undefined') {
        script = '<script src="{{script}}"></script>';
    }
    if(typeof properties['banner'] != 'undefined') {
        banner = 'background-image: url({{{banner}}});';
    }
    if(typeof properties['domain'] != 'undefined') {
        domain = properties['domain'].includes(':') ? '{{domain}}' : ('https://{{domain}}');
    }
    
    // automatically set title if it isn't set manually
    var result;
    if(typeof properties['title'] == 'undefined' && (result = (/<h1>(.*)<\/h1>/ig).exec(html))) {
        properties['title'] = result[1];
    }
    
    var pageHtml = `
<!doctype html>
<html>
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no">
<link rel="icon" href="{{logo}}">
<link rel="stylesheet" media="screen" href="{{stylesheet}}">
<meta property="og:type" content="website">
<meta property="og:title" content="{{title}}">
<link rel="canonical" href="${domain}/${layoutTitle}">
<title>{{title}}</title>
<style>
body > div.col-1:nth-of-type(2) {
${banner}
}
</style>
</head>
<body class="${layoutTitle.replace(/\//ig, ' ')} {{class}}">
${html}
${script}
</body>
</html>`;
    Mustache.parse(pageHtml);
    return Mustache.render(pageHtml, properties);
}

module.exports = wrapTemplate;


find known routes to sheets?


In [ ]:
var importer = require('../Core');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');

function overwriteProperties(l, properties, templates) {
    var urlParts = l.split('/');
    var key = urlParts[1];
    var fileName = key;

    return getTemplateProperties(key, properties, templates)
        .then(() => {
            // if rendering a page has url parts
            if(urlParts.length > 2) {
                // use the third argument to find the correct row in the data set
                var rowData = properties[key + '-data']
                    .filter(row => row['link'] == l || row['url'] == l);
                // handle multiple results for use with categories
                if(rowData.length == 0) {
                    rowData = {0: {}};
                    rowData[0][key + '-data'] = properties[key + '-data']
                        .filter(row => row[key] == l || row[key] == l.split('/')[2]);
                    // assign the matching value to the template property
                    rowData[0][key] = rowData[0][key + '-data'][0][key];
                }
                fileName = l.substr(1);
                return wrapTemplate(fileName, properties[key], Object.assign({}, properties, rowData[0]))
            }
            return wrapTemplate(fileName, properties[key], properties)
        })
        .then(page => {
            var returnObj = {};
            returnObj[fileName] = page;
            return returnObj;
        })
}

function collectRoutes(routes, properties, templates, rendered) {
    var local = routes.concat(properties[':render'] || [])
        .filter((cur, i, arr) => arr.indexOf(cur) == i && !cur.includes(':'))
        .filter(l => !rendered.includes(l) && typeof templates[l.split('/')[1]] != 'undefined')

    local.forEach(l => rendered[rendered.length] = l)
    
    return importer.runAllPromises(local
        // promise in series so there is no data collisions
        .map(l => resolve => overwriteProperties(l, properties, templates)
            .then(page => resolve(page))))
        .then(results => results.reduce((obj, r) => Object.assign(obj, r), {}))
}

module.exports = collectRoutes;



collect external content and resources?


In [ ]:
var {Readable} = require('stream');
var {JSDOM, XPathResult} = require('jsdom');
var importer = require('../Core');
var getArrayXPath = importer.import('get xpath array');

var regexToArray = (ex, str, i = 0) => {
    var co = []; var m; while ((m = ex.exec(str)) && co.push(m[i])); return co;
};

function collectExternalResources(page, rendered, routes) {
    
    // get all images and urls from template
    var dom = new JSDOM(page);
    
    var linksObjs = getArrayXPath('//a[@href]', dom.window.document.body);
    var links = linksObjs.map(l => l.getAttribute('href'));
    
    // TODO: convert images and add timestamps, add svg
    var imgObjs = getArrayXPath('//img[@src]', dom.window.document.body);
    var imgs = imgObjs.map(l => l.getAttribute('src'));
    
    // TODO: scan for urls and inline
    var stylesObjs = getArrayXPath('//link[@href]', dom.window.document.body);
    var styles = stylesObjs.map(l => l.getAttribute('href'));

    // TODO: add timestamps and inline
    var scriptsObjs = getArrayXPath('//script[@src]', dom.window.document.body);
    var scripts = scriptsObjs.map(l => l.getAttribute('src'));
    
    // TODO: add CSS imports
    var backgrounds = regexToArray(/url\((.*?)\)/ig, page, 1);
    
    var searches = imgs.concat(styles).concat(backgrounds).concat(scripts)
        .filter((cur, i, arr) => arr.indexOf(cur) == i && rendered.indexOf(cur) === -1);
    
    links.forEach(s => routes[routes.length] = s)
    searches.forEach(s => rendered[rendered.length] = s)
    
    // TODO: copy resource images to output directory
    var newPage = dom.window.document.documentElement.outerHTML
    var stream = new Readable();
    stream.push(newPage);
    stream.push(null);
    return Promise.resolve(stream);
}

module.exports = collectExternalResources;


collect google sheets resources?


In [ ]:
var importer = require('../Core');
var streamToGoogle = importer.import('upload files google cloud');
var copyFileBucket = importer.import('copy file bucket storage');
var collectExternalResources = importer.import('collect external content and resources');
var collectRoutes = importer.import('find known routes to sheets');

var timestamp = (new Date()).getTime();

// detect links and write out every part of the site
function collectTemplateResources(layoutTitle, page, properties, templates, bucketName, rendered, sTG) {
    console.log(`Rendering ${layoutTitle}`);
    
    if(!rendered) {
        rendered = [];
    }
    
    // if it is the first page in the template, rename it to index.html
    var template = layoutTitle.split('/')[0];
    if(templates[template] && templates[template].template
       && templates[template].template.properties.index === 0) {
        console.log(`Using ${layoutTitle} as index.html`);
        layoutTitle = 'index';
    }

    // TODO: add timestamps to generated content
    var routes = [];
    return collectExternalResources(page, rendered, routes)
        .then(stream => (sTG ? sTG : streamToGoogle)(layoutTitle + '.html', bucketName, stream, {
            contentType: 'text/html; charset=utf-8'
        }))
        .then(() => !sTG ? copyFileBucket(bucketName, layoutTitle + '.html') : Promise.resolve(false))
        .then(() => collectRoutes(routes, properties, templates, rendered))
        .then(pages => importer.runAllPromises(Object.keys(pages).map(fileName => resolve => 
            collectTemplateResources(fileName, pages[fileName], properties, templates, bucketName, rendered, sTG)
                .then(() => resolve()))))
        .then(() => rendered)
}

module.exports = collectTemplateResources;

test google sheets resources?


In [ ]:
var assert = require('assert');
var fs = require('fs');
var path = require('path');
var importer = require('../Core');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');
var findSimilarFile = importer.import('find similar file');

var DOWNLOAD_DIR = path.resolve(path.join(__dirname, '../../Downloads'));

// locally based utility for editing styles
function streamToOutput(fileUrl, bucketName, stream) {    
     return new Promise((resolve, reject) => {
        if(typeof stream == 'object') {
            var outputPath = path.join(path.resolve('./.output/'), fileUrl.replace(/\?.*/ig, ''));
            if(!fs.existsSync(path.dirname(outputPath))) {
                fs.mkdirSync(path.dirname(outputPath));
            }
            var writeStream = fs.createWriteStream(outputPath);
            stream.pipe(writeStream)
            .on("error", (err) => {
                reject(err);
            })
            .on('finish', () => {
                resolve(outputPath);
            });
        } else {
            var outputPath = path.join(path.resolve('./.output/'), path.basename(fileUrl.replace(/\?.*/ig, '')));
            var writeStream = fs.createWriteStream(outputPath);
            (fileUrl.includes('https://') ? https : http).get(fileUrl, response => {
                response.pipe(writeStream)
                .on("error", (err) => {
                    reject(err);
                })
                .on('finish', () => {
                    resolve(outputPath);
                });
            }).catch(e => console.log(e));
        }
     });
}

function outputTest(page, bucketName, key, templates, properties) {
    return collectTemplateResources(key, page, properties, templates, bucketName, false, streamToOutput)
        .then(resources => {
            console.log(resources);
            return Promise.all(resources.filter(r => !r.includes(':') && r.includes('.'))
                .map(r => {
                    // TODO: search notebooks for code so library can be saved to git
                    return findSimilarFile(path.basename(r), DOWNLOAD_DIR)
                        .then(f => {
                            if(f) {
                                return streamToOutput(
                                    f,
                                    bucketName,
                                    fs.createReadStream(path.join(DOWNLOAD_DIR, f)));
                            } else {
                                return Promise.resolve(false);
                            }
                        });
                }));
        });
}

function importTest(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var templates, key, properties = {};
    
    return getTemplates(docId)
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
                .then(() => wrapTemplate(key, properties[key], properties));
        })
        .then(page => {
            assert(page.length > 0, 'should have a page');
            return outputTest(page, domain, key, templates, properties);
        })
}

if(typeof describe !== 'undefined') {
    describe('output template resources', () => {

        it('should process at least one template', () => {
            var docsId = '1QeZ3WduNFmjtNf_Q3Xe5zAh8dzP_nn7nr-AQU0_hXg8';
            var bucketName = 'sheet-to-web.com';
            return importTest(docsId, bucketName);
        }).timeout(60000)
    })
    
}

module.exports = importTest;


get sheet identifier from link?


In [ ]:
var util = require('util');
var getInfo = importer.import('get google sheet info');
var getDataSheet = importer.import('google sheet array objects');
var addRow = importer.import('add row data google sheet');
var uuid = require('uuid/v1');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var project = 'spahaha-ea443';

function safeName(name) {
    return name.replace(/[^a-z\-]/ig, '-').substr(0, 40);
}

function getSheet(link) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var name, title;
    
    return getInfo(link)
        // return assigned subdomain
        .then(info => {
            console.log(info);
            name = safeName(info.properties.title.replace(/\s*(configuration|config)\s*/ig, '')) + '-' + uuid().substr(0, 5);
            title = info.properties.title;
            return getDataSheet(purchaseId, 'Purchases');
        })
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(match) {
                console.log(`Purchase ${docId} already exists: ${match.domain} or ${match.bucket}`);
                return Promise.resolve(match.domain || match.bucket);
            }
        
            console.log(`Adding product row for ${docId} ${name}`)
            return addRow(purchaseId, {
                timestamp: Date.now(),
                name: title,
                email: '',
                address: '',
                domain: '',
                bucket: name + '.sheet-to-web.com',
                project: project,
                sheet: docId
            })
                .then(() => name + '.sheet-to-web.com')
        })
}

module.exports = getSheet;

google sheet handler?


In [ ]:
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');

async function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return await getSheet(req.query['link'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports.handler = handler;


sheet marketing import?


In [ ]:
var getDataSheet = importer.import('google sheet array objects');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');

var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';

function importSheet(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var properties = {};
    
    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            return getTemplates(docId)
        })
        .then(t => {
            templates = t;
            key = Object.keys(templates).filter(k => templates[k].template)[0];
            return getTemplateProperties(key, properties, templates)
                .then(() => wrapTemplate(key, properties[key], properties));
        })
        .then(page => collectTemplateResources(key, page, properties, templates, domain))
        .then(resources => {
            console.log(resources);
            return resources;
        })
}

module.exports = importSheet;


sheet marketing import handler?


In [ ]:
var importer = require('../Core');
var importSheet = importer.import('sheet marketing import');

async function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return await importSheet(req.query['link'], req.query['domain'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports.handler = handler;


setup sheet backend?


In [ ]:
var getDataSheet = importer.import('google sheet array objects');
var purchaseId = '1kWjkjLGxQyzFUzRLBk3LpcjPW3UjcaF-PBMDX_3hZfM';
var addIP = importer.import('check dns');
var {
    insertBackendBucket,
    insertGlobalForward,
    updateUrlMap
} = importer.import('add google bucket web map');

var urlMap = 'web-map';

function setupBackend(link, domain) {
    var docId = link.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var project;

    return getDataSheet(purchaseId, 'Purchases')
        .then(purchases => {
            var match = purchases.filter(p => p.sheet == docId)[0];
            if(!match || domain !== match.domain && domain !== match.bucket) {
                throw new Error(`Sheet ${docId} doesn't match domain ${domain}`);
            }
            project = match.project;
            return addIP(project, domain);
        })
        .then(ip => insertGlobalForward(project, ip, urlMap, domain))
        .then(() => insertBackendBucket(project, domain))
        .then(() => updateUrlMap(project, urlMap, domain))
}

module.exports = setupBackend;

sheet backend handler?


In [ ]:
var importer = require('../Core');
var setupBackend = importer.import('setup sheet backend');

function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    return setupBackend(req.query['link'], req.query['domain'])
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}
    
module.exports.handler = handler;


contact us handler?

In [ ]:
var importer = require('../Core');
var addRow = importer.import('add row data google sheet');

var contactId = '16b1Z0sQkYhtMUfP7qMRL3vRBPWQsRbSlntURkMqWGX0';
var redirect = 'https://www.shopilluminati.com/contact';

function handler(req, res) {
    res.set('Access-Control-Allow-Origin', '*');
    console.log(`Adding contact row for ${req.body['name']}`)
    return addRow(contactId, {
        timestamp: Date.now(),
        name: req.body['name'],
        email: req.body['email'],
        subject: req.body['subject'],
        message: req.body['message'],
        responded: 0,
    })
        .then(r => res.redirect(redirect))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

    
module.exports.handler = handler;


package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
